In [ ]:
from datasets import load_dataset
issues_dataset = load_dataset('lewtun/github-issues',split='train')

In [ ]:
issues_dataset

In [ ]:
issues_dataset = issues_dataset.filter(lambda x: (x['is_pull_request'] == False and len(x['comments']) > 0))

In [ ]:
issues_dataset

In [ ]:
columns = issues_dataset.column_names

In [ ]:
columns

In [ ]:
columns_to_keep = ['title','body','comments','html_url']

In [ ]:
coulmns_to_remove = set(columns_to_keep).symmetric_difference(columns)

In [ ]:
issues_dataset = issues_dataset.remove_columns(coulmns_to_remove)

In [ ]:
issues_dataset

In [ ]:
issues_dataset.set_format('pandas')

In [ ]:
df = issues_dataset[:]

In [ ]:
df

In [ ]:
df['comments'][0].tolist()

In [ ]:
comments_df = df.explode('comments',ignore_index=True)

In [ ]:
comments_df.head(4)

In [ ]:
from datasets import Dataset

In [ ]:
comments_dataset = Dataset.from_pandas(comments_df)

In [ ]:
comments_dataset

In [ ]:
comments_dataset = comments_dataset.map(lambda x: {'comment_length': len(x['comments'].split())})

In [ ]:
comments_dataset = comments_dataset.filter(lambda x: x['comment_length'] > 15)

In [ ]:
comments_dataset

In [ ]:
def concatenate_text(examples):
    return {
        'text': examples['title']
        + ' \n '
        + examples['body']
        + ' \n '
        + examples['comments']
    }

In [ ]:
comments_dataset = comments_dataset.map(concatenate_text)

In [ ]:
comments_dataset[0]

In [ ]:
from transformers import AutoTokenizer,AutoModel

In [ ]:
model_ckt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckt)

In [ ]:
model = AutoModel.from_pretrained(model_ckt)

In [ ]:
import torch

In [ ]:
device = torch.device('cuda')

In [ ]:
model = model.to(device)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:,0]

In [ ]:

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list,padding=True,truncation=True,return_tensors='pt'
    )
    encoded_input = {k:v.to(device) for k,v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
embbeding = get_embeddings(comments_dataset['text'][0])

In [ ]:
embbeding.shape

In [ ]:
embbeding_dataset = comments_dataset.map(lambda x: {'embeddings': get_embeddings(x['text']).detach().cpu().numpy()[0]})

In [ ]:
embbeding_dataset.add_faiss_index(column='embeddings')

In [ ]:
question = 'How can i load a dataset offline'

In [ ]:
question_embedding = get_embeddings([question]).cpu().detach().numpy()

In [ ]:
question_embedding.shape

In [ ]:
scores,samples = embbeding_dataset.get_nearest_examples('embeddings',question_embedding,k=100)

In [ ]:
scores

In [ ]:
import pandas as pd

In [ ]:
samples_df = pd.DataFrame.from_dict(samples)

In [ ]:
samples_df['scores'] = scores

In [ ]:
samples_df.sort_values('scores',ascending=False,inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()